In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
import random

In [0]:
def initialise(X, n, random_state):
  np.random.seed(random_state)
  w = [None for _ in range(len(n))]
  b = [None for _ in range(len(n))]

  for i in range(len(n)):
    if i == 0:
      n_minus_1 = X.shape[1]
    else:
      n_minus_1 = n[i-1]

    w[i] = np.random.randn(n_minus_1, n[i]) * np.sqrt(2. / n_minus_1)
    b[i] = np.random.randn(1, n[i])

  return w, b

In [0]:
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

In [0]:
def relu(z):
  return np.maximum(z, 0)

In [0]:
def relu_prime(z):
    return (z > 0) * 1

In [0]:
def get_activation_func(activation_name):
  if activation_name == 'relu':
    return relu
  elif activation_name == 'sigmoid':
    return sigmoid

In [0]:
def forward_prop(X, w, b, n, activation):

  z = [None for _ in range(len(n))]
  a = [None for _ in range(len(n))]

  for i in range(len(n)):
    if i == 0:
      a_minus_1 = X
    else:
      a_minus_1 = a[i-1]

    z[i] = np.dot(a_minus_1, w[i]) + b[i] 
    a[i] = get_activation_func(activation[i])(z[i])

  return z, a

In [0]:
def backward_prop(X, y, w, z, a):
  dw = [None for _ in range(len(w))]
  db = [None for _ in range(len(w))]
  dz = [None for _ in range(len(w))]

  for i in reversed(range(len(w))):
    if i == len(w) - 1:
      dz[i] = a[i] - y
    else:
      dz[i] = np.dot(dz[i+1], w[i+1].T) * relu_prime(z[i])
    
    if i == 0:
      a_minus_1 = X    
    else:
      a_minus_1 = a[i-1]

    dw[i] = np.dot(a_minus_1.T, dz[i]) / X.shape[0]    
    db[i] = np.sum(dz[i], axis = 0, keepdims= True) / X.shape[0]
    
  return dz, dw, db

In [0]:
def update(alpha, w, b, dw, db, n):
  for i in range(len(n)):
    w[i] = w[i] - (alpha * dw[i])
    b[i] = b[i] - (alpha * db[i])
    
  return w, b

In [0]:
def loss(y, y_hat):
  L = -((y * np.log(y_hat + 1e-15)) + ((1 - y ) * np.log(1 - y_hat + 1e-15))) / y.shape[0]
  return np.nansum(L)

In [0]:
import h5py

def load_dataset():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])

    classes = np.array(test_dataset["list_classes"][:])
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [0]:
def train(X_train, y_train, X_test, y_test, epochs, alpha, n):
  loss_array = []
  w, b = initialise(X_train, n , random_state=0)

  for i in range(epochs+1):
    z, a = (forward_prop(X_train, w, b, n, activation))
    dz, dw, db = backward_prop(X_train, y_train, w, z, a)
    w, b = update(alpha, w, b, dw, db, n)
    loss_array.append(loss(y_train,a[-1]))

    if i % (epochs / 10) == 0:
      pred_train = np.where(a[-1] > 0.5, 1, 0)
      _, a_test = (forward_prop(X_test, w, b, n, activation))
      pred_test = np.where(a_test[-1] > 0.5, 1, 0)

      print('Epoch: {}\tLoss: {:.5f}\t\tTraining Acc: {:.5f}\tTest Acc: {:.5f}'.
            format(i, loss(y_train,a[-1]), accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)))

  return w, b, loss_array

In [0]:
def predict(X_test, y_test, w, b, n, activation):
  _, a = (forward_prop(X_test, w, b, n, activation))
  pred = np.where(a[-1] > 0.5, 1, 0)

  return accuracy_score(y_test, pred)

In [0]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = load_dataset()

X_train = train_set_x_orig
y_train = train_set_y_orig

X_train = X_train.reshape(X_train.shape[0], -1) / 255
y_train = y_train.reshape(y_train.shape[0], 1)

X_test = test_set_x_orig
y_test = test_set_y_orig

X_test = X_test.reshape(X_test.shape[0], -1) / 255
y_test = y_test.reshape(y_test.shape[0], 1)

In [16]:
alpha = 5e-3
epochs = 2000

activation = ['relu','relu', 'sigmoid']

n = [10, 3, 1]

w, b, loss_array = train(X_train, y_train, X_test, y_test, epochs, alpha, n)

Epoch: 0	Loss: 0.68697		Training Acc: 0.66029	Test Acc: 0.34000
Epoch: 200	Loss: 0.43266		Training Acc: 0.75120	Test Acc: 0.42000
Epoch: 400	Loss: 0.30902		Training Acc: 0.90909	Test Acc: 0.62000
Epoch: 600	Loss: 0.21357		Training Acc: 0.95694	Test Acc: 0.70000
Epoch: 800	Loss: 0.13955		Training Acc: 0.98565	Test Acc: 0.70000
Epoch: 1000	Loss: 0.08861		Training Acc: 0.98565	Test Acc: 0.74000
Epoch: 1200	Loss: 0.06414		Training Acc: 0.98565	Test Acc: 0.74000
Epoch: 1400	Loss: 0.04775		Training Acc: 0.98565	Test Acc: 0.72000
Epoch: 1600	Loss: 0.03723		Training Acc: 0.99522	Test Acc: 0.74000
Epoch: 1800	Loss: 0.03032		Training Acc: 0.99522	Test Acc: 0.72000
Epoch: 2000	Loss: 0.02585		Training Acc: 0.99522	Test Acc: 0.70000


In [17]:
pred = predict(X_test, y_test, w, b, n, activation)

print('Test accuracy: ', pred)

Test accuracy:  0.7
